In [1]:
import _pickle as cp
import numpy as np

### Dataset Loader

In [4]:
def load_dataset(name, get_xy = False):
    X,y = cp.load(open(name,'rb'))
    if get_xy:
        return X, y
    N, D = X.shape
    N_train = int(.8 * N)
    N_test = N - N_train
    
    X_train = X[:N_train]
    y_train = y[:y_train]
    X_test = X[N_train:]
    y_test = y[N_train:]
    
    return X_train, y_train, X_test, y_test

### Task 1 - Visualisation

In [6]:
import matplotlib.pyplot as plt

In [20]:
def visualise(name):
    X, y = load_dataset('winequality-red.pickle', True)
    unique, counts = np.unique(y.astype(int), return_counts=True)
    print(unique)
    print(counts)
    plt.bar(range(min(unique),max(unique)+1), counts)
    plt.show()

### Task 2 - Predict average as solution and find MSE

In [21]:
def mean_predictor(y_train, y_test):
    y_avg = np.mean(y_train)
    mse = np.mean(np.power(y_test - y_avg, 2))
    print('The average is '+str(y_avg))
    print('The mse is '+str(mse))

### Task 3 - Linear Regression

In [24]:
import os

def scaler(X, reset = False):
    if os.path.exists('musigma.pickle') and not reset:
        print('musigma exists!, using values')
        mean, std = cp.load(open('musigma.pickle','rb'))
        X = (X-mean)/std
    else:
        mean = np.mean(X)
        std = np.std(X)
        musigma = (mean, std)
        cp.dump(musigma,open('musigma.pickle', 'wb'))
        X = (X-mean)/std
        
    return X


def linear_regression(X_train, y_train, X_test, y_test):
    
    x_t = X_train[:int(len(X_train)*.8)]
    x_v = X_train[int(len(X_train)*.8):]
    
    x_train_scaled = scaler(x_t)
    w = np.pinv(x_train_scaled.T * x_train_scaled)* x_train_scaled.T * y_train
    
    x_validation_scaled = scaler(x_v)
    
    y_pred = x_validation_scaled * w
    mse = np.mean(np.power(y_test - y_pred, 2))
    print('The mean squared error is '+str(mse))
    np.save('weights.npy', w)
    print('Weights saved')